# Gather Hourly Traffic Data <br>
*From statens vegvesen API: https://www.vegvesen.no/trafikkdata/api*

Import necessary packages:

In [7]:
import os
import json
import requests
import time
import random
from tqdm import tqdm  
import pandas as pd
from datetime import datetime
# Display all columns in the df
pd.set_option('display.max_columns', None)

Set paths etc.:

In [2]:
# define the base path where the file should be saved
base_path = '/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData'
# define the file name
file_name = "all_IDs.json"
# combine the base path and file name to create a valid file path
file_path = os.path.join(base_path, file_name)

### First query (get traffic point ID's):

In [3]:
# define the endpoint of the API
endpoint = "https://www.vegvesen.no/trafikkdata/api/?query=%0A%20%20%20%20query%7B%0A%20%20%20%20%20%20trafficRegistrationPoints(searchQuery%3A%20%7BroadCategoryIds%3A%20%5BE%5D%20%7D)%20%7B%0A%20%20%20%20%20%20%20%20id%0A%20%20%20%20%20%20%20%20name%0A%20%20%20%20%20%20%20%20location%20%7B%0A%20%20%20%20%20%20%20%20%20%20coordinates%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20latLon%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20lat%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20lon%0A%20%20%20%20%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%7D%0A%20%20%20%20%7D%0A%20%20%0A%20"

# define the GraphQL query for fetching ID's of registration points (only Europe roads)
query = """
{
  trafficRegistrationPoints(searchQuery: {roadCategoryIds: [E ,R, F], registrationFrequency: CONTINUOUS}) {
    id
  }
}
"""

# make a POST request to the API
response = requests.post(endpoint, json={"query": query})

# check the status code of the response
if response.status_code == 200:
    # if the request was successful, get the data from the response
    data = response.json()

    # open the file for writing
    with open(file_path, "w") as file:
        # write the data to the file in JSON format
        json.dump(data, file)

Store query number 1 and convert ID's to list for inserting in query number 2:

In [3]:
# define the file path of the JSON file
file_path = '/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/all_IDs.json'

# open the file for reading
with open(file_path, "r") as file:
    # load the data from the file into a Python object
    data = json.load(file)

# create an empty list for the registration point ID's
registration_point_ids = []

# iterate over the data returned by the GraphQL query
for item in data["data"]["trafficRegistrationPoints"]:
    # get the ID of the current registration point
    id = item["id"]

    # add the ID to the list of registration point ID's
    registration_point_ids.append(id)

### Second query (get traffic volume for every hour in a one-week period): <br>
*NB! This query must be run **twice** because it only possible to get 100 hours per query.* <br>
*The dataframes must therefore be merged together later on.*

In [5]:
# define the endpoint of the API
endpoint = """https://www.vegvesen.no/trafikkdata/api/?query=%7B%0A%20%20trafficData(trafficRegistrationPointId%3A%20%'DISTINCT_ID'%22)%20%7B%0A%20%20%20%20volume%20%7B%0A%20%20%20%20%20%20byHour(%0A%20%20%20%20%20%20%20%20from%3A%20%222019-10-24T12%3A00%3A00%2B02%3A00%22%0A%20%20%20%20%20%20%20%20to%3A%20%222019-10-24T14%3A00%3A00%2B02%3A00%22%0A%20%20%20%20%20%20)%20%7B%0A%20%20%20%20%20%20%20%20edges%20%7B%0A%20%20%20%20%20%20%20%20%20%20node%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20from%0A%20%20%20%20%20%20%20%20%20%20%20%20to%0A%20%20%20%20%20%20%20%20%20%20%20%20total%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20volumeNumbers%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20volume%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20coverage%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20percentage%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%7D%0A%20%20%20%20%7D%0A%20%20%7D%0A%7D%0A"""

data_1 = []
with tqdm(total=len(registration_point_ids)) as pbar:     # HUSK Å FJERNE [:300] FOR Å FÅ MED ALLE TRAFFIC POINTS
  for registration_point_id in registration_point_ids:    # HUSK Å FJERNE [:300] FOR Å FÅ MED ALLE TRAFFIC POINTS
    time.sleep(random.uniform(1, 3))

    # Change to: "2018-09-23T23:00:00+02:00"
    query = """ {
  trafficData(trafficRegistrationPointId: "%s") {
    volume {
      byHour(
        from: "2018-09-17T00:00:00+02:00"
        to: "2018-09-20T12:00:00+02:00"  
      ) {
        edges {
          node {
            from
            to
            total {
              volumeNumbers {
                volume
              }
              coverage {
                percentage
              }
            }
          }
        }
      }
    }
  }
}
    """ % registration_point_id

    response = requests.post(endpoint, json={"query": query})

    if response.status_code == 200:
      volume_data_1 = response.json()

      # Add the registration_point_id to the volume_data dictionary
      volume_data_1["registration_point_id"] = registration_point_id

      data_1.append(volume_data_1)

      pbar.update(1)

100%|██████████| 3125/3125 [1:49:01<00:00,  2.09s/it]


*Second run:*

In [6]:
# define the endpoint of the API
endpoint = """https://www.vegvesen.no/trafikkdata/api/?query=%7B%0A%20%20trafficData(trafficRegistrationPointId%3A%20%'DISTINCT_ID'%22)%20%7B%0A%20%20%20%20volume%20%7B%0A%20%20%20%20%20%20byHour(%0A%20%20%20%20%20%20%20%20from%3A%20%222019-10-24T12%3A00%3A00%2B02%3A00%22%0A%20%20%20%20%20%20%20%20to%3A%20%222019-10-24T14%3A00%3A00%2B02%3A00%22%0A%20%20%20%20%20%20)%20%7B%0A%20%20%20%20%20%20%20%20edges%20%7B%0A%20%20%20%20%20%20%20%20%20%20node%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20from%0A%20%20%20%20%20%20%20%20%20%20%20%20to%0A%20%20%20%20%20%20%20%20%20%20%20%20total%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20volumeNumbers%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20volume%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20coverage%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20percentage%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%7D%0A%20%20%20%20%7D%0A%20%20%7D%0A%7D%0A"""

data_2 = []
with tqdm(total=len(registration_point_ids)) as pbar:     # HUSK Å FJERNE [:300] FOR Å FÅ MED ALLE TRAFFIC POINTS
  for registration_point_id in registration_point_ids:    # HUSK Å FJERNE [:300] FOR Å FÅ MED ALLE TRAFFIC POINTS
    time.sleep(random.uniform(1, 3))

    # Change to: "2018-09-23T23:00:00+02:00"
    query = """ {
  trafficData(trafficRegistrationPointId: "%s") {
    volume {
      byHour(
        from: "2018-09-20T12:00:00+02:00"
        to: "2018-09-23T23:00:00+02:00"  
      ) {
        edges {
          node {
            from
            to
            total {
              volumeNumbers {
                volume
              }
              coverage {
                percentage
              }
            }
          }
        }
      }
    }
  }
}
    """ % registration_point_id

    response = requests.post(endpoint, json={"query": query})

    if response.status_code == 200:
      volume_data_2 = response.json()

      # Add the registration_point_id to the volume_data dictionary
      volume_data_2["registration_point_id"] = registration_point_id

      data_2.append(volume_data_2)

      pbar.update(1)

100%|██████████| 3125/3125 [1:49:41<00:00,  2.11s/it]  


Store data collected from query number 2:

In [7]:
# define the path and filename for the JSON file
filename_1 = '/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/hourlyVolume_1.json'
# write the data to the JSON file
with open(filename_1, "w") as f:
    json.dump(data_1, f, indent=2) # usually indent = 4, but ok 

# define the path and filename for the JSON file
filename_2 = '/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/hourlyVolume_2.json'
# write the data to the JSON file
with open(filename_2, "w") as f:
    json.dump(data_2, f, indent=2) 

Open data collected from query number 2:

In [8]:
with open('/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/hourlyVolume_1.json', 'r') as file:
    json_data_1 = json.load(file)

with open('/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/hourlyVolume_2.json', 'r') as file:
    json_data_2 = json.load(file)

### def flatten_json <br>
*Takes the data gathered from query 2 (stored as a json-file), flattening the data and converts it to a pandas dataframe.*

In [9]:
def flatten_json(json_data):
    flat_list = []
    for item in json_data:
        edges = item.get("data", {}).get("trafficData", {}).get("volume", {}).get("byHour", {}).get("edges", [])
        traffic_registration_point_id = item.get("registration_point_id")

        data_count = 0
        temp_list = []
        for edge in edges:
            node = edge.get("node", {})
            total = node.get("total", {})

            if total is None:
                continue

            volume_numbers = total.get("volumeNumbers") or {}
            coverage = total.get("coverage") or {}

            if volume_numbers.get("volume") is None:
                continue

            from_datetime = datetime.fromisoformat(node.get("from").replace("+02:00", ""))
            to_datetime = datetime.fromisoformat(node.get("to").replace("+02:00", ""))

            flat_dict = {
                "traffic_point_id": traffic_registration_point_id,
                "year": from_datetime.year,
                "month": from_datetime.month,
                "day": from_datetime.day,
                "from_hour": from_datetime.strftime("%H:%M"),
                "to_hour": to_datetime.strftime("%H:%M"),
                "volume": volume_numbers.get("volume"),
                "coverage_percentage": coverage.get("percentage")
            }
            data_count += 1
            temp_list.append(flat_dict)

        if data_count >= 83:    # 83 er riktig tall egentlig            # NB! Denne burde egentlig være "if data_count == 168:", men det ser ut til at det kun er mulig å hente ut 100 timer på ett punkt om gangen.
                                            # data_count == 168 fordi 24 timer i 7 dager (24*7) = 168
            flat_list.extend(temp_list)

    return flat_list


**Read warning in function above.**

Run function, check output:

In [10]:
flat_data_1 = flatten_json(json_data_1)
flat_data_2 = flatten_json(json_data_2)
# Convert flattened data to DataFrame
volume_df_1 = pd.DataFrame(flat_data_1)
volume_df_2 = pd.DataFrame(flat_data_2)
display(volume_df_1)#[:50]
display(volume_df_2)#[:50]

,traffic_point_id,year,month,day,from_hour,to_hour,volume,coverage_percentage
0,65823V1668921,2018,9,17,00:00,01:00,20,100.0
1,65823V1668921,2018,9,17,01:00,02:00,6,100.0
2,65823V1668921,2018,9,17,02:00,03:00,1,100.0
3,65823V1668921,2018,9,17,03:00,04:00,0,100.0
4,65823V1668921,2018,9,17,04:00,05:00,4,100.0
...,...,...,...,...,...,...,...,...
162115,10795V320297,2018,9,20,07:00,08:00,755,100.0
162116,10795V320297,2018,9,20,08:00,09:00,657,100.0
162117,10795V320297,2018,9,20,09:00,10:00,453,100.0
162118,10795V320297,2018,9,20,10:00,11:00,455,100.0


,traffic_point_id,year,month,day,from_hour,to_hour,volume,coverage_percentage
0,65823V1668921,2018,9,20,12:00,13:00,68,100.0
1,65823V1668921,2018,9,20,13:00,14:00,53,100.0
2,65823V1668921,2018,9,20,14:00,15:00,69,100.0
3,65823V1668921,2018,9,20,15:00,16:00,56,100.0
4,65823V1668921,2018,9,20,16:00,17:00,44,100.0
...,...,...,...,...,...,...,...,...
162260,10795V320297,2018,9,23,18:00,19:00,511,100.0
162261,10795V320297,2018,9,23,19:00,20:00,525,100.0
162262,10795V320297,2018,9,23,20:00,21:00,357,100.0
162263,10795V320297,2018,9,23,21:00,22:00,256,100.0


### Merge the two dataframes:

In [11]:
# Concatenate the two dataframes
merged_df = pd.concat([volume_df_1, volume_df_2], ignore_index=True)

# Group by traffic_point_id and filter based on the number of data points (168 hours)
completeVolume_df = merged_df.groupby('traffic_point_id').filter(lambda x: len(x) >= 167)

# Reset the index of the filtered dataframe
completeVolume_df.reset_index(drop=True, inplace=True)


Check the merged dataframe - looks good:

In [12]:
completeVolume_df#[:60]

,traffic_point_id,year,month,day,from_hour,to_hour,volume,coverage_percentage
0,65823V1668921,2018,9,17,00:00,01:00,20,100.0
1,65823V1668921,2018,9,17,01:00,02:00,6,100.0
2,65823V1668921,2018,9,17,02:00,03:00,1,100.0
3,65823V1668921,2018,9,17,03:00,04:00,0,100.0
4,65823V1668921,2018,9,17,04:00,05:00,4,100.0
...,...,...,...,...,...,...,...,...
317462,10795V320297,2018,9,23,18:00,19:00,511,100.0
317463,10795V320297,2018,9,23,19:00,20:00,525,100.0
317464,10795V320297,2018,9,23,20:00,21:00,357,100.0
317465,10795V320297,2018,9,23,21:00,22:00,256,100.0


Save dataframe as CSV-file in directory:

In [13]:
# Specify the file path and name
output_file_path = "/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/completeAllHourlyVolume.csv"

# Convert the dataframe to a CSV file and save it to the specified path
completeVolume_df.to_csv(output_file_path, index=False)


In [5]:
print(completeVolume_df['traffic_point_id'].nunique() / len(registration_point_ids))

0.60832


In [4]:
completeVolume_df = pd.read_csv('/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/completeAllHourlyVolume.csv')
completeVolume_df

,traffic_point_id,year,month,day,from_hour,to_hour,volume,coverage_percentage
0,65823V1668921,2018,9,17,00:00,01:00,20,100.0
1,65823V1668921,2018,9,17,01:00,02:00,6,100.0
2,65823V1668921,2018,9,17,02:00,03:00,1,100.0
3,65823V1668921,2018,9,17,03:00,04:00,0,100.0
4,65823V1668921,2018,9,17,04:00,05:00,4,100.0
...,...,...,...,...,...,...,...,...
317462,10795V320297,2018,9,23,18:00,19:00,511,100.0
317463,10795V320297,2018,9,23,19:00,20:00,525,100.0
317464,10795V320297,2018,9,23,20:00,21:00,357,100.0
317465,10795V320297,2018,9,23,21:00,22:00,256,100.0


### Pivot the dataframe <br> 
*This is to display hourly volume on the y-axis (not sure if necessary)*

In [8]:
# Create a new column 'timestamp' combining year, month, day, and from_hour
completeVolume_df['timestamp'] = completeVolume_df['year'].astype(str) + '-' + completeVolume_df['month'].astype(str).str.zfill(2) + '-' + completeVolume_df['day'].astype(str).str.zfill(2) + ' ' + completeVolume_df['from_hour']

# Pivot the DataFrame using 'traffic_point_id' as index and 'timestamp' as columns
pivoted_df = completeVolume_df.pivot_table(index='traffic_point_id', columns='timestamp', values='volume')

# Reset index and fill missing values with 0 (if any)
pivoted_df.reset_index(inplace=True)
pivoted_df.fillna(0, inplace=True)

# Display pivoted dataframe
pivoted_df

timestamp,traffic_point_id,2018-09-17 00:00,2018-09-17 01:00,2018-09-17 02:00,2018-09-17 03:00,2018-09-17 04:00,2018-09-17 05:00,2018-09-17 06:00,2018-09-17 07:00,2018-09-17 08:00,2018-09-17 09:00,2018-09-17 10:00,2018-09-17 11:00,2018-09-17 12:00,2018-09-17 13:00,2018-09-17 14:00,2018-09-17 15:00,2018-09-17 16:00,2018-09-17 17:00,2018-09-17 18:00,2018-09-17 19:00,2018-09-17 20:00,2018-09-17 21:00,2018-09-17 22:00,2018-09-17 23:00,2018-09-18 00:00,2018-09-18 01:00,2018-09-18 02:00,2018-09-18 03:00,2018-09-18 04:00,2018-09-18 05:00,2018-09-18 06:00,2018-09-18 07:00,2018-09-18 08:00,2018-09-18 09:00,2018-09-18 10:00,2018-09-18 11:00,2018-09-18 12:00,2018-09-18 13:00,2018-09-18 14:00,2018-09-18 15:00,2018-09-18 16:00,2018-09-18 17:00,2018-09-18 18:00,2018-09-18 19:00,2018-09-18 20:00,2018-09-18 21:00,2018-09-18 22:00,2018-09-18 23:00,2018-09-19 00:00,2018-09-19 01:00,2018-09-19 02:00,2018-09-19 03:00,2018-09-19 04:00,2018-09-19 05:00,2018-09-19 06:00,2018-09-19 07:00,2018-09-19 08:00,2018-09-19 09:00,2018-09-19 10:00,2018-09-19 11:00,2018-09-19 12:00,2018-09-19 13:00,2018-09-19 14:00,2018-09-19 15:00,2018-09-19 16:00,2018-09-19 17:00,2018-09-19 18:00,2018-09-19 19:00,2018-09-19 20:00,2018-09-19 21:00,2018-09-19 22:00,2018-09-19 23:00,2018-09-20 00:00,2018-09-20 01:00,2018-09-20 02:00,2018-09-20 03:00,2018-09-20 04:00,2018-09-20 05:00,2018-09-20 06:00,2018-09-20 07:00,2018-09-20 08:00,2018-09-20 09:00,2018-09-20 10:00,2018-09-20 11:00,2018-09-20 12:00,2018-09-20 13:00,2018-09-20 14:00,2018-09-20 15:00,2018-09-20 16:00,2018-09-20 17:00,2018-09-20 18:00,2018-09-20 19:00,2018-09-20 20:00,2018-09-20 21:00,2018-09-20 22:00,2018-09-20 23:00,2018-09-21 00:00,2018-09-21 01:00,2018-09-21 02:00,2018-09-21 03:00,2018-09-21 04:00,2018-09-21 05:00,2018-09-21 06:00,2018-09-21 07:00,2018-09-21 08:00,2018-09-21 09:00,2018-09-21 10:00,2018-09-21 11:00,2018-09-21 12:00,2018-09-21 13:00,2018-09-21 14:00,2018-09-21 15:00,2018-09-21 16:00,2018-09-21 17:00,2018-09-21 18:00,2018-09-21 19:00,2018-09-21 20:00,2018-09-21 21:00,2018-09-21 22:00,2018-09-21 23:00,2018-09-22 00:00,2018-09-22 01:00,2018-09-22 02:00,2018-09-22 03:00,2018-09-22 04:00,2018-09-22 05:00,2018-09-22 06:00,2018-09-22 07:00,2018-09-22 08:00,2018-09-22 09:00,2018-09-22 10:00,2018-09-22 11:00,2018-09-22 12:00,2018-09-22 13:00,2018-09-22 14:00,2018-09-22 15:00,2018-09-22 16:00,2018-09-22 17:00,2018-09-22 18:00,2018-09-22 19:00,2018-09-22 20:00,2018-09-22 21:00,2018-09-22 22:00,2018-09-22 23:00,2018-09-23 00:00,2018-09-23 01:00,2018-09-23 02:00,2018-09-23 03:00,2018-09-23 04:00,2018-09-23 05:00,2018-09-23 06:00,2018-09-23 07:00,2018-09-23 08:00,2018-09-23 09:00,2018-09-23 10:00,2018-09-23 11:00,2018-09-23 12:00,2018-09-23 13:00,2018-09-23 14:00,2018-09-23 15:00,2018-09-23 16:00,2018-09-23 17:00,2018-09-23 18:00,2018-09-23 19:00,2018-09-23 20:00,2018-09-23 21:00,2018-09-23 22:00
0,00000V1702725,5,4,0,2,2,21,132,433,436,469,624,801,862,868,952,1243,1268,1003,936,857,525,252,45,16,12,4,2,5,5,18,108,399,399,402,582,683,764,756,902,1062,1197,928,809,699,446,237,41,20,8,0,0,2,5,16,122,426,419,474,647,794,879,952,942,1223,1258,978,903,810,531,264,59,19,10,5,2,5,0,20,128,449,435,462,650,814,952,1030,965,1268,1274,995,992,874,626,268,56,10,12,4,10,1,4,22,121,375,415,469,849,984,1031,1082,1044,1069,790,566,469,396,310,211,37,21,9,3,3,2,3,4,20,37,88,335,809,1373,1625,1665,1846,1756,1513,1229,846,437,101,41,23,15,17,9,5,1,2,1,4,3,11,33,37,85,133,157,235,212,228,198,136,106,96,52,32
1,00000V1702751,243,82,51,56,131,624,1721,3305,3064,2398,2562,2859,2757,2929,3260,4415,3929,2779,2496,2062,1497,1024,526,390,162,90,56,64,114,550,1697,3307,3020,2235,2359,2481,2494,2634,3181,4257,3962,2847,2296,1855,1486,1026,532,383,160,74,45,59,122,500,1727,3220,3010,2257,2376,2651,2924,2968,3325,4378,4018,3033,2444,2108,1499,1027,568,342,156,81,67,70,125,503,1635,3175,2915,2272,2427,2730,3112,3161,3391,4443,4175,3139,2672,2255,1692,1137,605,514,317,106,80,63,129,537,1564,2820,2701,2268,2585,2902,3175,3360,3392,3209,2230,1

Store new pivoted dataframe:

In [9]:
# Specify the file path and name
output_file_path = "/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/completeAllHourlyVolume_pivoted.csv"

# Convert the dataframe to a CSV file and save it to the specified path
pivoted_df.to_csv(output_file_path, index=False)